# Chapter 5: Libraries

Een belangrijke troef van Python is de uitgebreide set bibliotheken die de basisfunctionaliteit uitbreiden. De toolboxen bestrijken een breed terrein: van optimalisatie, over beeldverwerking tot economie. De handleidingen zijn, zoals altijd, duidelijk en uitgebreid. In dit hoofdstuk maken we kennis met twee bibliotheken, OpenCV en Open3D. De uitleg is met opzet kort gehouden, zodat de leerling zelf kan zoeken naar de relevante functies en syntaxen. 

## Intro

### Download files

In [ ]:
!git clone https://github.com/delima87/Ch4Data.git

### Read video

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Ch4Data/VideoKROOK.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width="640" height="480" controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


# 4.1 Computer Vision library (OpenCV)
Python heeft uitgebreide bibliotheken voor beeldverwerking. In deze sectie zullen we zien hoe we afbeeldingen of video's kunnen lezen of schrijven en er basisbewerkingen op kunnen uitvoeren die gebruikt worden in fotogrammetrische toepassingen. Als voorbeeld gebruiken we de opnames die gemaakt zijn door de onderzoeksgroep {Constructiemechanica en Bouwmaterialen} als onderdeel van een onderzoek naar de invloed van voetgangers op bruggen. Het onderzoek in kwestie beoogt de interactie tussen voetgangers en de brug te onderzoeken. Daartoe is het belangrijk om de positie van de lopende mensen in de tijd te kennen, en een eerste stap bestaat uit het identificeren van de verschillende kleuren helmen van mensen. 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

frameId = 100 # Change this value to pick a frame

cap = cv2.VideoCapture('/content/Ch4Data/VideoKROOK.mp4')
frameCounter =0
img2Process = np.zeros((720,512,3), np.uint8)
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    frameCounter += 1
    if frameCounter == frameId:
        img2Process = cv2.resize(frame, (720,512), interpolation = cv2.INTER_AREA)
        break
  # Break the loop
  else:
    break

img2Process_rgb = cv2.cvtColor(img2Process, cv2.COLOR_BGR2RGB)
plt.imshow(img2Process_rgb)

## 4.1.1 Light green caps
De eerste stap is het detecteren van mogelijke voetgangers. Dit gebeurt onafhankelijk in alle beelden afzonderlijk. We gebruiken de petten die de proefpersonen in de video dragen. Deze petten hebben specifieke kleuren die duidelijk herkenbaar zijn en opvallen tegen de achtergrond. Jouw taak is om de lichtgroene petten in elk beeld te detecteren. Je gebruikt hiervoor een eenvoudige kleursegmentatie: een pixel kan behoren tot een lichtgroen petje als het volgende geldt voor de RGB-kleuren:
$$
\textrm{R} \in [120-180] \nonumber\\
\textrm{G} \in [180-230] \nonumber\\
\textrm{B} \in [110-160]
$$
Pixels die niet aan deze eis voldoen behoren zeker niet tot een lichtgroene kap. Het is aan jou om de volgende stappen uit te voeren, met behulp van de functionaliteit van OpenCV:

- Gebruik het bestand `VideoKROOK.mp4}`, dat is opgenomen op de voetgangersbrug aan de Krook in Gent.
- Lees elk frame van de video en converteer naar uint8.
- Elk frame is een 3D matrix: ${\bf I}(x,y,c)$ waar $x$ en $y$ de afbeeldingscoordinaten zijn en $c$ de kleur is (1=R, 2= G, 3=B). In andere woorden ${\bf I}(120,251,3)$ is de blauwe waarde van het coordinaat $(120, 251)$.
- Voor elk frame bekijk je nu de RGB-waarden en maak je een binaire afbeelding met 1 op de pixels die voldoen aan de kleurfilter en 0 op de pixels die niet voldoen.
- Visualiseer enkele van deze beelden (bijvoorbeeld met de functie `cv2.imshow()`). Een voorbeeld van afbeelding 100 is te zien hieronder.

Frame 100             |  Detected caps
:-------------------------:|:-------------------------:
![](frame100.jpg)  |  ![](detectionresult100.png)

In [ ]:
# set the correct limits for r,g,b
lR = 0
lG = 0
lB = 0
hR = 0
hG = 0
hB = 0

In [ ]:
low = (lR, lG, lB)
high = (hR, hG, hB)
im_rgb = cv2.cvtColor(img2Process, cv2.COLOR_BGR2RGB)
bin_img = cv2.inRange(im_rgb, low, high)
plt.imshow(bin_img, cmap='Greys',  interpolation='nearest')

## 4.1.2 Morphological Operations
Je ziet dat de lichtgroene dopjes worden gedetecteerd, maar dat ook veel verkeerde pixels als 'light green cap' worden geïdentificeerd. Om dit op te lossen, passen we enkele morfologische operatoren toe op de binaire beelden. Het gaat om *erode* en *dilate* filters die de gebieden respectievelijk verkleinen en vergroten.

- Breid de vorige code uit en werk op de binaire beelden met de detecties.
- Begin met een erodeerfilter met een rechthoek van 2x2. Pas dat toe op de binaire afbeelding uit de vorige stap en kijk naar `cv2.erode()` en `cv2.dilate()`.
- Pas dan een dilatatiefilter toe op het resultaat met een rechthoek van 16x16.
- Voer tenslotte een erodeerfilter uit op dat resultaat met een rechthoek van 4x4.
- Visualiseer een aantal van deze beelden `cv2.imshow()`. Een voorbeeld van afbeelding 100 is te zien hieronder


Frame 100             |  Detected caps
:-------------------------:|:-------------------------:
![](frame100.jpg)  |  ![](morphologicalresult100.png)

In [ ]:
#erode kernel 2 by 2
kernel = np.ones((2,2), np.uint8)
img_erosion = cv2.erode(bin_img, kernel, iterations=1)
#dilate kernel 16 by 16
kernel2 = np.ones((16,16), np.uint8)#define kernel 2
img_dilation = cv2.dilate(img_erosion, kernel2, iterations=1)
#erode with kernel 4 by 4
#YOUR CODE >

# < YOUR CODE
plt.imshow(img_dilation, cmap='Greys',  interpolation='nearest')

# 4.2 Pointcloud library (Open3D)
De technologische ontwikkeling op het gebied van geomatica en landmeetkunde gaat razendsnel. Een belangrijke innovatie die de laatste jaren ingang heeft gevonden, is het gebruik van laserscanners, zowel op land als op mobiele voertuigen. Dergelijke scanners hebben het voordeel dat zij in korte tijd zeer grote datasets kunnen opnemen die de omgeving rond het apparaat in kaart brengen, in de vorm van puntenwolken.

Een groot nadeel van puntwolken van laserscanners is de omvang van de gegevens. Een enkele scan bevat al snel miljoenen punten, zodat de opname van een straat of een stad al snel onbeheersbaar wordt. Er bestaan natuurlijk goede softwareprogramma's die de grafische kaarten (GPU) van uw computer gebruiken om de puntenwolken weer te geven, maar de interactie en het meten in de puntenwolken blijft een probleem.

Daarom richten steeds meer mensen zich op het automatisch segmenteren en verwerken van puntenwolken. Er bestaan verschillende python-bibliotheken die verschillende functies bevatten die je hiervoor kunt gebruiken. 

## Installing Open3D

In [ ]:
!pip install open3d

In [ ]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go

## 4.2.1 Visualize Point Clouds
Open3D is een python-bibliotheek voor het importeren en verwerken van puntenwolken. Het bevat een reeks algoritmen om niet alleen 3D-gegevens te visualiseren, maar ook om high-level 3D-toepassingen te ontwikkelen. De volledige documentatie van de verschillende Open3D modules is te vinden in deze link: [Open3d Documentation](http://www.open3d.org/docs/release/).

In [ ]:
def showPCD(pcd):
  points = np.asarray(pcd.points)
  colors = None
  if pcd.has_colors():
      colors = np.asarray(pcd.colors)
  elif pcd.has_normals():
      colors = (0.5, 0.5, 0.5) + np.asarray(pcd.normals) * 0.5
  fig = go.Figure(
  data=[
    go.Scatter3d(
      x=points[:,0], y=points[:,1], z=points[:,2],
      mode='markers',
      marker=dict(size=1, color=colors)
  )
  ],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False)
  )
  )
  )
  fig.show()

In [ ]:
pcd = o3d.io.read_point_cloud('/content/Ch4Data/cloud_2.ply')
showPCD(pcd)

## 4.2.2 Plane Fitting

Aangezien puntenwolken die met laserscanners worden gegenereerd gewoonlijk vrij dicht zijn, is het computationeel duur om metingen uit te voeren. Een mogelijke oplossing is het inpassen van geometrische primitieven in de puntenwolk. De belangrijkste primitieve is de eenvoudigste 3D vorm: een vlak. Open3D gebruikt hiervoor de functie `segment_plane()`. Je kan verschillende parameters doorgeven aan deze functie, maar in zijn eenvoudigste vorm heeft hij alleen een puntenwolk, een maximale afstand, het aantal minimale punten en iteraties nodig. De functie berekent dan het best passende vlak (d.w.z. het vlak met de meeste punten in de puntenwolk). Naast het vlak geeft deze functie ook de inliers of de punten die in het vlak liggen. 

### Eerste vlak

- Laad een puntenwolk naar keuze in.
- Gebruik `segment_plane()` om het best passende vlak in de puntenwolk te passen. Kies zelf een goede waarde voor de maximale afstand `distance_threshold`). Stel de andere parameters in met deze waarden:  `ransac_n = 3, num_iterations=500`.
- Bekijk het model dat je terugkrijgt. Wat zouden deze getallen betekenen?
- Gebruik de functie `select_by_index()` om alleen de inliers van dit vlak uit de oorspronkelijke puntenwolk te selecteren.
- Visualiseer de inliers en de puntenwolk. Gebruik de functie `paint_uniform\color()` om de uitschieters in te kleuren.

In [ ]:
# Fitting 1st plane
plane_model, inliers = pcd.segment_plane(distance_threshold=0.08,ransac_n=3,num_iterations=500)
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
# visualizing extracted plane
showPCD(inlier_cloud+ outlier_cloud)

### Andere vlakken
Zodra het eerste (dominante) vlak is aangebracht, kunt u op zoek gaan naar het op één na beste vlak. Volg hiervoor de volgende deze stappen:
- Nadat het dominante vlak is gedetecteerd, wordt een nieuwe puntenwolk gemaakt waarin de inliers van het eerste vlak zijn verwijderd. Gebruik daarvoor de functie `select_by_index()` en de uitschieters. filter de verkregen inliers en stel `invert = True`.
- Voer nu dezelfde functie ({\tt segment\_plane}) uit op deze puntenwolk en vind het op één na beste vlak.
- Herhaal stappen één en twee om het derde, vierde of vijfde vlak te vinden.
- Visualiseer de resultaten door de gebieden telkens een andere kleur te geven.

In [ ]:

# Plane 2


In [ ]:
# Plane 3


In [ ]:
# Plane 4

In [ ]:
# Plane 5

## 4.2.3 Cutting Lines
Voor veel toepassingen zijn we niet direct geïnteresseerd in oppervlakken, maar in hun snijpunten. Denk maar aan de GRB waar bijvoorbeeld de gevellijn wordt bepaald als het snijpunt tussen de gevel en de grond. Of bijvoorbeeld de nok van een dak waar de twee schuine vlakken elkaar snijden In de laatste oefening bepaal je de nok van zo'n dak.

### Plane extraction on gable roof

In [ ]:
#read gable roof
gable_roof = '/content/Ch4Data/gableRoof2.ply'
roof = o3d.io.read_point_cloud(gable_roof)
showPCD(roof)

Bepaal het snijpunt van de 2 relevante dominante vlakken om de nok van het dak te bepalen. Daartoe herhalen we een beetje wiskunde. Stel je hebt twee vlakken ${\tt \Pi_1}$ and ${\tt \Pi_2}$ met vergelijking:
$$
{\tt \Pi_1}: a_1X+b_1Y+c_1Z+d_1 = 0\nonumber \\
{\tt \Pi_2}: a_2X+b_2Y+c_2Z+d_2 = 0
$$

Hoe bereken je het snijpunt van deze twee vlakken? De vergelijking van een rechte kan parametrisch geschreven worden:
$$
{\tt M} = {\tt M_0} + \lambda {\tt D}\nonumber \\
\begin{bmatrix}X\\Y\\Z\end{bmatrix} = \begin{bmatrix}X_0\\Y_0\\Z_0\end{bmatrix} + \lambda\begin{bmatrix}X_d\\Y_d\\Z_d\end{bmatrix}
$$
waardat ${\tt M_0}$ is een punt op de lijn en ${\tt D}$ is de richting van de lijn. Deze richting is gemakkelijk te berekenen. We weten immers dat het snijpunt van de twee vlakken in beide vlakken moet liggen. Dat betekent dat de richting ervan loodrecht moet staan op de normalen van beide vlakken:
$$
{\tt D} \bot {\tt n_1}\\
{\tt D} \bot {\tt n_2}
$$
Zo kan ${\tt D}$ bepaald worden als het vectorproduct van ${\tt n_1}$ en ${\tt n_2}$:
$$
{\tt D} = {\tt n_1} \times {\tt n_2}
$$
Je ziet een illustratie hier:

![](cuttingplanes.png)

We hoeven alleen maar ${\tt M_0}$ (een willekeurig punt op de lijn) te bepalen. We kunnen dit eenvoudig als volgt doen:
- Bepaal 1 coördinaat in de buurt van de lijn, Bijvoorbeeld een $(X)$ coördinaat: $X = X_0$
- Als $X_0$ bekend is, bepaal dan $Y_0$ en $Z_0$, wetende dat het punt op de secans moet liggen, en dus aan beide vergelijkingen moet voldoen. Stel de twee vergelijkingen op en los het systeem op.


"Het volledige algoritme wordt dan:
- Bepaal de normalen van de vlakken ${\tt n_1}$ en ${\tt n_2}$.
- Bepaal de richting ${\tt D} = {\tt n_1} \times {\tt n_2}$ (tip: `numpy.cross()`).
- Kies een coördinaat ${X_0}$.
- Stel het stelsel ($Y_0$ and $Z_0$) op en los op.
- Teken nu het snijpunt door $\\lambda$ te variëren. Als ${\\tt n_1}$ en ${\\tt n_2}$ eenheidsvectoren zijn, is $\\lambda$ een afstand in meters. Afhankelijk van de keuze van je startpunt ($X_0$) moet je $\\lambda$ positieve en/of negatieve waarden geven. 
- Om de lijn te tekenen gebruik je de functie open3D `geometry.LineSet()`, waar je de punten moet invoeren die de lijn vormen en de array van verbindingen. Een voorbeeld van het gebruik van deze functie vindt u in deze [link](http://www.open3d.org/docs/0.7.0/tutorial/Basic/visualization.html)"
      

In [ ]:
#YOUR CODE >

# Extract plane1


# Plane2


# Visualise

# < YOUR CODE


Punt $x0$ ligt op de snijlijn

In [ ]:
x0 =103831
#YOUR CODE >


# < YOUR CODE